<a href="https://colab.research.google.com/github/sergioemacedor-prog/clase-13-Macedo/blob/main/Copia_de_examen.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# ***Predicción del precio de la gasolina con Streamlit***

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [3]:
# Load the CSV data into a pandas DataFrame named 'df'.
df = pd.read_csv('/content/Preciosgasolina.csv')
# Display the first few rows of the DataFrame to inspect its structure and content.
display(df.head())

,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,Precios_promedio_diarios_y_mensuales_en_estaciones_de_servicio
Fecha,Aguascalientes,BajaCalifornia,BCaliforniaSur,Campeche,Chiapas,Chihuahua,CDMX,Coahuila,Colima,Durango,Guanajuato,Guerrero,Hidalgo,Jalisco,Michoacan,Morelos,Mexico,Nayarit,NuevoLeon,Oaxaca,Puebla,Quertaro,QuintanaRoo,SanLuisPotosi,Sinaloa,Sonora,Tabasco,Tamaulipas,Tlaxcala,Veracruz,Yucatan,Zacatecas
2017/01,16.19,16.07,16.11,15.67,15.86,16.04,16.32,15.8,16.06,16.25,16.1,16.03,15.85,16.46,16.22,15.91,16.16,16.31,16.28,15.86,15.86,16.04,15.89,15.85,16.12,16.12,15.57,15.16,15.79,15.52,15.44,16.19
2017/02,16.18,16.04,16.07,15.65,15.85,14.26,16.29,15.85,16.02,16.21,16.06,16.02,15.89,16.44,16.22,15.88,16.13,16.29,16.26,15.86,15.86,16.02,15.87,15.82,16.09,16.14,15.56,15.19,15.8,15.53,15.41,16.21
2017/03,16.1,15.85,16,15.56,15.76,14.76,16.21,15.71,15.95,16.15,16,15.98,15.78,16.36,16.14,15.81,16.05,16.23,16.18,15.77,15.77,15.95,15.79,15.76,16.02,15.77,15.49,14.15,15.7,15.44,15.34,16.12
2017/04,16.12,15.87,16.02,15.59,15.78,14.74,16.2,15.62,15.99,16.17,16.03,15.99,15.81,16.36,16.16,15.84,16.06,16.25,16.18,15.79,15.79,15.98,15.81,15.78,16.04,15.67,15.52,13.87,15.72,15.47,15.36,16.14


# 1. ETL: Limpia, transforma y estructura los datos

In [4]:
#Codificar estado
# Load the dataset, skipping the first row and setting the header to the first row (which was originally the second row).
df = pd.read_csv('/content/Preciosgasolina.csv', skiprows=[0], header=0)

# Rename the first column to 'Fecha'
df = df.rename(columns={df.columns[0]: 'Fecha'})

# Create 'Año' and 'Mes' columns from the 'Fecha' column.
# Convert the 'Fecha' column to datetime objects for easier extraction of year and month.
df['Fecha'] = pd.to_datetime(df['Fecha'], errors='coerce') # Use errors='coerce' to handle potential parsing issues
# Drop rows where Fecha is not valid after coercion
df.dropna(subset=['Fecha'], inplace=True)
# Extract the year into a new 'Año' column.
df['Año'] = df['Fecha'].dt.year
# Extract the month into a new 'Mes' column.
df['Mes'] = df['Fecha'].dt.month

# Inspect column names to understand the current structure.
print("Column names:", [col for col in df.columns])

# Remove duplicate columns to ensure data integrity.
df = df.loc[:, ~df.columns.duplicated()]


# Define the identifier variables ('Fecha', 'Año', 'Mes') and value variables (states) for melting the DataFrame.
id_vars = ['Fecha', 'Año', 'Mes']
# Assume all other columns are states and should be transformed into rows.
# Exclude the initial descriptive column if it was not removed earlier
value_vars = [col for col in df.columns if col not in id_vars and col != 'Precios_promedio_diarios_y_mensuales_en_estaciones_de_servicio']


# Melt the DataFrame to transform state columns into rows, making the data long format.
# 'Estado' will be the new column for state names, and 'Precio' will hold the gasoline prices.
df_melted = df.melt(id_vars=id_vars, value_vars=value_vars, var_name='Estado', value_name='Precio')

# Convert the 'Precio' column to a numeric data type, coercing any errors to NaN.
df_melted['Precio'] = pd.to_numeric(df_melted['Precio'], errors='coerce')

# Drop rows where 'Precio' is NaN, as these represent failed conversions and cannot be used for modeling.
df_melted.dropna(subset=['Precio'], inplace=True)


# Encode the 'Estado' column into numerical categories.
# This is necessary for using the state information as a feature in the machine learning model.
df_melted['Estado_Encoded'] = df_melted['Estado'].astype('category').cat.codes

# Display the first few rows of the melted and processed DataFrame to verify the changes.
display(df_melted.head())

Column names: ['Fecha', 'Aguascalientes', 'BajaCalifornia', 'BCaliforniaSur', 'Campeche', 'Chiapas', 'Chihuahua', 'CDMX', 'Coahuila', 'Colima', 'Durango', 'Guanajuato', 'Guerrero', 'Hidalgo', 'Jalisco', 'Michoacan', 'Morelos', 'Mexico', 'Nayarit', 'NuevoLeon', 'Oaxaca', 'Puebla', 'Quertaro', 'QuintanaRoo', 'SanLuisPotosi', 'Sinaloa', 'Sonora', 'Tabasco', 'Tamaulipas', 'Tlaxcala', 'Veracruz', 'Yucatan', 'Zacatecas', 'Año', 'Mes']


/tmp/ipython-input-4231630182.py:10: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df['Fecha'] = pd.to_datetime(df['Fecha'], errors='coerce') # Use errors='coerce' to handle potential parsing issues


,Fecha,Año,Mes,Estado,Precio,Estado_Encoded
0,2017-01-01,2017,1,Aguascalientes,16.19,0
1,2017-02-01,2017,2,Aguascalientes,16.18,0
2,2017-03-01,2017,3,Aguascalientes,16.10,0
3,2017-04-01,2017,4,Aguascalientes,16.12,0
4,2017-05-01,2017,5,Aguascalientes,16.03,0


# 2. Construye un modelo predictivo con una regresión lineal múltiple para estimar el precio de la gasolina en función del estado, mes y año

In [5]:
import statsmodels.api as sm
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score
import joblib

In [6]:
#Variables independientes y dependientes
X = df_melted[['Estado_Encoded', 'Año', 'Mes']]
y = df_melted['Precio']

In [7]:
#Dividir
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [8]:
#Entrenar el modelo
# Add a constant to the independent variable for the OLS model
# X_train = sm.add_constant(X_train)
# X_test = sm.add_constant(X_test)

# Train the OLS model
model = sm.OLS(y_train, sm.add_constant(X_train)).fit()

# Print the model summary
print(model.summary())

# Make predictions on the test set
y_pred = model.predict(sm.add_constant(X_test))

                            OLS Regression Results                            
Dep. Variable:                 Precio   R-squared:                       0.808
Model:                            OLS   Adj. R-squared:                  0.808
Method:                 Least Squares   F-statistic:                     3595.
Date:                Sun, 21 Sep 2025   Prob (F-statistic):               0.00
Time:                        02:45:34   Log-Likelihood:                -4007.6
No. Observations:                2560   AIC:                             8023.
Df Residuals:                    2556   BIC:                             8047.
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
                     coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------
const          -1978.1213     19.263   -102.

In [9]:
# Train the Linear Regression model using scikit-learn.
from sklearn.linear_model import LinearRegression

# Instantiate a LinearRegression model.
modelo = LinearRegression()
# Train the model using the training data (X_train and y_train).
modelo.fit(X_train, y_train)

LinearRegression()

In [10]:
#Evaluar el modelo
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print(f"Mean Squared Error: {mse}")
print(f"R-squared: {r2}")

Mean Squared Error: 1.349549595372777
R-squared: 0.8051444853829416


In [11]:
from sklearn.metrics import mean_squared_error, r2_score
import numpy as np

y_pred = modelo.predict(X_test)
print("R2:", r2_score(y_test, y_pred))
print("RMSE:", np.sqrt(mean_squared_error(y_test, y_pred)))

R2: 0.8051444853829584
RMSE: 1.16170116440187


In [12]:
#Estima el precio de la gasolina para un estado y una fecha

def estimate_gas_price(estado, año, mes):
  """
  Estimates the price of gasoline for a given state, year, and month.

  Args:
    estado: The name of the state (string).
    año: The year (integer).
    mes: The month (integer).

  Returns:
    The estimated gasoline price (float).
  """
  # Convert state string to encoded numerical value
  try:
    estado_encoded = df_melted[df_melted['Estado'] == estado]['Estado_Encoded'].iloc[0]
  except IndexError:
    return "Error: Invalid state provided."

  # Create input DataFrame for prediction
  input_data = pd.DataFrame({'Estado_Encoded': [estado_encoded], 'Año': [año], 'Mes': [mes]})

  # Add a constant to the input data to match the training data
  # input_data = sm.add_constant(input_data, prepend=True)


  # Predict the price
  prediction = modelo.predict(input_data)

  return prediction[0]

# Example usage (for testing purposes)
estimated_price = estimate_gas_price('Jalisco', 2024, 10)
# print(f"Estimated price in Jalisco in October 2024: {estimated_price:.2f}")

In [13]:
# Print the estimated price
print(f"Estimated price in Jalisco in December 2025: {estimated_price:.2f}")

Estimated price in Jalisco in December 2025: 23.67


In [14]:
# Call the estimate_gas_price function with example values
estimated_price = estimate_gas_price('Jalisco', 2025, 12)

# Print the estimated price
print(f"Estimated price in Jalisco in December 2025: {estimated_price:.2f}")

Estimated price in Jalisco in December 2025: 24.81


# 3. Aplicación en STREAMLIT

In [23]:

%%writefile Gasolina.py
import numpy as np
import streamlit as st
import pandas as pd
import joblib

# Cargar modelo entrenado
modelo = joblib.load("Gasolina.pkl")

# Cargar dataset para obtener lista de estados
df = pd.read_csv("gasolina_limpia.csv")
estados = sorted(df["estado"].unique())

# Interfaz
st.title("📊 Estimador del precio de gasolina")

estado = st.selectbox("Selecciona un estado:", estados)
mes = st.slider("Selecciona un mes:", 1, 12, 6)
año = st.slider("Selecciona un año:", int(df["año"].min()), int(df["año"].max()), 2023)

# Convertir estado a código
estado_cod = df[df["estado"] == estado]["estado_cod"].iloc[0]

# Predicción
if st.button("Predecir precio"):
    prediccion = modelo.predict([[estado_cod, mes, año]])
    st.metric("Precio estimado de gasolina", f"${prediccion[0]:.2f} MXN")


Overwriting Gasolina.py


In [ ]:
# Save the trained model to a file
joblib.dump(modelo, "Gasolina.pkl")

In [26]:
!npm install localtunnel
!npm audit fix --force

⠙⠹⠸
up to date, audited 23 packages in 601ms
⠼
⠼3 packages are looking for funding
⠼  run `npm fund` for details
⠼
2 high severity vulnerabilities

To address all issues (including breaking changes), run:
  npm audit fix --force

Run `npm audit` for details.
⠼npm warn using --force Recommended protections disabled.
⠙⠹⠸⠼npm warn audit Updating localtunnel to 1.8.3, which is a SemVer major change.
⠼⠴⠦⠧⠇⠏⠋⠙⠹⠸⠼⠴⠦⠧⠇⠏⠋⠙⠹⠸⠼⠴⠦⠧⠇⠏⠋⠙⠹⠸⠼⠴⠦⠧⠇⠏⠋⠙⠹⠸⠼⠴⠦⠧⠇⠏npm warn deprecated cryptiles@2.0.5: This version has been deprecated in accordance with the hapi support policy (hapi.im/support). Please upgrade to the latest version to get the best features, bug fixes, and security patches. If you are unable to upgrade at this time, paid support is available for older versions (hapi.im/commercial).
⠏⠋npm warn deprecated sntp@1.0.9: This module moved to @hapi/sntp. Please make sure to switch over as this distribution is no longer supported and may contain bugs and critical security issues.
⠋⠙npm warn deprecated h

In [ ]:
!streamlit run Gasolina.py &>/content/logs.txt & npx localtunnel --port 8501

⠙your url is: https://tired-falcons-relax.loca.lt
